In [ ]:
# Importar librerías necesarias
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import os
import datetime
import json

In [ ]:
video_dir = 'C:\\Tesis\\TestErgo\\muestra'

In [ ]:
def load_videos(video_dir):
    videos = []
    for filename in os.listdir(video_dir):
        if filename.endswith(".mp4"):
            videos.append(os.path.join(video_dir, filename))
    return videos

In [ ]:
# Cargar videos
videos = load_videos(video_dir)

In [ ]:
# Función para extraer frames de los videos
def extract_frames(video_path, interval=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % interval == 0:
            frames.append(frame)
        count += 1
    cap.release()
    return frames


In [ ]:
# Obtener y mostrar los nombres de todos los videos
for video_path in videos:
    video_name = os.path.basename(video_path)
    print(video_name)

In [ ]:
# Inicializar MediaPipe para detección de puntos clave
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Función para calcular el ángulo entre tres puntos
def calculate_angle(a, b, c, d=None):
    if d is None:
        # Calcula el ángulo entre a, b y c
        a = np.array(a)  # Primer punto
        b = np.array(b)  # Segundo punto (vértice)
        c = np.array(c)  # Tercer punto

        radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
        angle = np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = 360.0 - angle

        return angle
    else:
        # Calcula el ángulo entre a, b, c y d
        a = np.array(a)  # Primer punto
        b = np.array(b)  # Segundo punto (vértice)
        c = np.array(c)  # Tercer punto
        d = np.array(d)  # Cuarto punto

        radians = np.arctan2(d[1] - c[1], d[0] - c[0]) - np.arctan2(b[1] - a[1], b[0] - a[0])
        angle = np.abs(radians * 180.0 / np.pi)

        if angle > 180.0:
            angle = 360.0 - angle

        return angle


In [ ]:
# Función para detectar puntos clave usando MediaPipe
def detect_keypoints(video_path):
    cap = cv2.VideoCapture(video_path)
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
    keypoints_list = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)
        if results.pose_landmarks:
            keypoints = [(lm.x, lm.y, lm.z) for lm in results.pose_landmarks.landmark]
            keypoints_list.append(keypoints)
        else:
            keypoints_list.append([])  # Agregar una lista vacía si no se detectan keypoints

    cap.release()
    pose.close()
    return keypoints_list

In [ ]:
# Función para analizar videos masivos y guardar resultados en un solo JSON
def analyze_videos_in_folder(video_folder, output_json_path):
    all_analysis_results = []
    
    for video_file in os.listdir(video_folder):
        if video_file.endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(video_folder, video_file)
            video_name = os.path.splitext(video_file)[0]
            
            keypoints_list = detect_keypoints(video_path)
            cap = cv2.VideoCapture(video_path)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            cap.release()

            analysis_results = analyze_keypoints(keypoints_list, video_name, fps)
            all_analysis_results.extend(analysis_results)
            draw_keypoints_and_angles(video_path, analysis_results, output_folder, output_video_folder, json_folder)
    
    combined_json_path = os.path.join(json_folder, 'Resultado_analisis_combinados.json')
    with open(output_json_path, 'w') as json_file:
        json.dump(all_analysis_results, json_file, indent=4)

    print(f"Resultados del análisis exportados a: {output_json_path}")

# Ejemplo de uso:
#video_folder = 'C:\\Tesis\\TestErgo\\muestra'
video_folder = video_dir #'C:\\Tesis\\TestErgo\\VIDEOS MAQUILAS'
output_json_path = 'C:\\Tesis\\TestErgo\\analisis_videos.json'

analyze_videos_in_folder(video_folder, output_json_path)


In [57]:
def analyze_keypoints(video_path):
    keypoints_list = detect_keypoints(video_path)
    fps = None
    analysis_results = []

    for frame_idx, keypoints in enumerate(keypoints_list):
        second = frame_idx // fps if fps else 0

        if not keypoints:
            analysis_results.append({
                'segundo': second,
                'frame': frame_idx,
                'video_name': None,
                'angulo_hombro_izquierdo': None,
                'angulo_del_hombro_derecho': None,
                'angulo_codo_izquierdo': None,
                'angulo_codo_derecho': None,
                'angulo_de_muneca_izquierda': None,
                'angulo_de_muneca_derecha': None,
                'angulo_mano_izquierdo': None,
                'angulo_mano_derecho': None,
                'posicion_hombro_izquierdo': None,
                'posicion_hombro_derecho': None,
                'posicion_codo_izquierdo': None,
                'posicion_codo_derecho': None,
                'posicion_muneca_izquierda': None,
                'posicion_muneca_derecha': None,
                'posicion_mano_izquierda': None,
                'posicion_mano_derecha': None,
                'keypoints': []
            })
            continue

        # Extraer puntos clave izquierdos
        left_shoulder = keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        left_elbow = keypoints[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        left_wrist = keypoints[mp_pose.PoseLandmark.LEFT_WRIST.value]
        left_hand = keypoints[mp_pose.PoseLandmark.LEFT_INDEX.value]

        # Extraer puntos clave derechos
        right_shoulder = keypoints[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        right_elbow = keypoints[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        right_wrist = keypoints[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        right_hand = keypoints[mp_pose.PoseLandmark.RIGHT_INDEX.value]

        # Calcular ángulos izquierdos
        left_shoulder_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        left_elbow_angle = calculate_angle(left_elbow, left_wrist, left_hand)
        left_wrist_angle = calculate_angle(left_elbow, left_wrist, left_hand)
        left_hand_angle = calculate_angle(left_elbow, left_wrist, left_hand)

        # Calcular ángulos derechos
        right_shoulder_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        right_elbow_angle = calculate_angle(right_elbow, right_wrist, right_hand)
        right_wrist_angle = calculate_angle(right_elbow, right_wrist, right_hand)
        right_hand_angle = calculate_angle(right_elbow, right_wrist, right_hand)

        # Si no se ha establecido aún el FPS, se establece ahora
        if fps is None:
            cap = cv2.VideoCapture(video_path)
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            cap.release()

        analysis_results.append({
            'segundo': second,
            'frame': frame_idx,
            'analisis_video': video_path,
            'angulo_hombro_izquierdo': left_shoulder_angle,
            'angulo_del_hombro_derecho': right_shoulder_angle,
            'angulo_codo_izquierdo': left_elbow_angle,
            'angulo_codo_derecho': right_elbow_angle,
            'angulo_de_muneca_izquierda': left_wrist_angle,
            'angulo_de_muneca_derecha': right_wrist_angle,
            'angulo_mano_izquierdo': left_hand_angle,
            'angulo_mano_derecho': right_hand_angle,
            'posicion_hombro_izquierdo': left_shoulder,
            'posicion_hombro_derecho': right_shoulder,
            'posicion_codo_izquierdo': left_elbow,
            'posicion_codo_derecho': right_elbow,
            'posicion_muneca_izquierda': left_wrist,
            'posicion_muneca_derecha': right_wrist,
            'posicion_mano_izquierda': left_hand,
            'posicion_mano_derecha': right_hand,
            'keypoints': keypoints
        })

    return analysis_results, fps

In [58]:
def draw_keypoints_and_angles(video_path, output_folder, output_video_folder, json_folder):
    analysis_results, fps = analyze_keypoints(video_path)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error al abrir el video: {video_path}")
        return
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    video_frame_folder = os.path.join(output_folder, video_name)
    output_video_name = f"{video_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
    output_video_path = os.path.join(output_video_folder, output_video_name)
    json_filename = f"{video_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    json_path = os.path.join(json_folder, json_filename)

    # Crear carpetas si no existen
    if not os.path.exists(video_frame_folder):
        os.makedirs(video_frame_folder)
    if not os.path.exists(output_video_folder):
        os.makedirs(output_video_folder)
    if not os.path.exists(json_folder):
        os.makedirs(json_folder)

    # Crear el video marcado
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    for i in range(len(analysis_results)):
        ret, frame = cap.read()
        if not ret:
            break

        keypoints = analysis_results[i]['keypoints']
        left_shoulder_angle = analysis_results[i]['angulo_hombro_izquierdo']
        right_shoulder_angle = analysis_results[i]['angulo_del_hombro_derecho']
        left_elbow_angle = analysis_results[i]['angulo_codo_izquierdo']
        right_elbow_angle = analysis_results[i]['angulo_codo_derecho']
        left_wrist_angle = analysis_results[i]['angulo_de_muneca_izquierda']
        right_wrist_angle = analysis_results[i]['angulo_de_muneca_derecha']
        left_hand_angle = analysis_results[i]['angulo_mano_izquierdo']
        right_hand_angle = analysis_results[i]['angulo_mano_derecho']

        # Dibujar puntos clave y conexiones con el color original
        if keypoints:
            for j, point in enumerate(keypoints):
                cv2.circle(frame, (int(point[0] * frame.shape[1]), int(point[1] * frame.shape[0])), 5, (0, 255, 0), -1)

                # Añadir texto con información relevante junto a cada punto
                if j == mp_pose.PoseLandmark.LEFT_SHOULDER.value:
                    cv2.putText(frame, f'Angulo del hombro izquierdo: {int(left_shoulder_angle) if left_shoulder_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_SHOULDER.value:
                    cv2.putText(frame, f'Angulo del hombro derecho: {int(right_shoulder_angle) if right_shoulder_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.LEFT_ELBOW.value:
                    cv2.putText(frame, f'Angulo del codo izquierdo: {int(left_elbow_angle) if left_elbow_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_ELBOW.value:
                    cv2.putText(frame, f'Angulo del codo derecho: {int(right_elbow_angle) if right_elbow_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.LEFT_WRIST.value:
                    cv2.putText(frame, f'Angulo de la muneca izquierda: {int(left_wrist_angle) if left_wrist_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_WRIST.value:
                    cv2.putText(frame, f'Angulo de la muneca derecha: {int(right_wrist_angle) if right_wrist_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.LEFT_INDEX.value:
                    cv2.putText(frame, f'Angulo de la mano izquierda: {int(left_hand_angle) if left_hand_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
                elif j == mp_pose.PoseLandmark.RIGHT_INDEX.value:
                    cv2.putText(frame, f'Angulo de la mano derecha: {int(right_hand_angle) if right_hand_angle else 0}',
                                (int(point[0] * frame.shape[1]) + 10, int(point[1] * frame.shape[0]) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

        # Guardar el frame en la carpeta de salida con el nombre del video seguido de su numeración
        output_frame_path = os.path.join(video_frame_folder, f'{video_name}_frame_{frame_count:04d}.png')
        cv2.imwrite(output_frame_path, frame)
        frame_count += 1

        # Guardar el frame en el video marcado
        out.write(frame)

    cap.release()
    out.release()

    # Guardar el archivo JSON con los resultados del análisis
    with open(json_path, 'w') as json_file:
        json.dump(analysis_results, json_file, indent=4)

    print(f"Frames exportados a: {video_frame_folder}")
    print(f"Video exportado a: {output_video_path}")
    print(f"JSON exportado a: {json_path}")
    return video_frame_folder, output_video_path, json_path


In [59]:
video_path = videos[0]
output_folder = 'C:\\Tesis\\TestErgo\\resultados'
output_video_folder = 'C:\\Tesis\\TestErgo\\videoMarcado'
json_folder = 'C:\\Tesis\\TestErgo\\archivos_json'

draw_keypoints_and_angles(video_path, output_folder, output_video_folder, json_folder)

c:\Tesis\TesisOCRA\TesisCheckListOCRA\python\venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Frames exportados a: C:\Tesis\TestErgo\resultados\maquila006
Video exportado a: C:\Tesis\TestErgo\videoMarcado\maquila006_20240714_233305.mp4
JSON exportado a: C:\Tesis\TestErgo\archivos_json\maquila006_20240714_233305.json


('C:\\Tesis\\TestErgo\\resultados\\maquila006',
 'C:\\Tesis\\TestErgo\\videoMarcado\\maquila006_20240714_233305.mp4',
 'C:\\Tesis\\TestErgo\\archivos_json\\maquila006_20240714_233305.json')